In [1]:
import logging
import pathlib
import sys
from typing import Optional

import numpy as np
import pandas as pd
import toml
from copairs.map import run_pipeline
from pycytominer import feature_select

# imports src
sys.path.append("../")
from src import utils

# setting up logger
logging.basicConfig(
    filename="map_analysis_testing.log",
    level=logging.DEBUG,
    format="%(levelname)s:%(asctime)s:%(name)s:%(message)s",
)

## Helper functions
Set of helper functions to help out throughout the notebook

In [2]:
## Helper function


def shuffle_meta_labels(
    dataset: pd.DataFrame, target_col: str, seed: Optional[int] = 0
) -> pd.DataFrame:
    """shuffles labels or values within a single selected column

    Parameters
    ----------
    dataset : pd.DataFrame
        dataframe containing the dataset

    target_col : str
        Column to select in order to conduct the shuffling

    seed : int
        setting random seed

    Returns
    -------
    pd.DataFrame
        shuffled dataset

    Raises
    ------
    TypeError
        raised if incorrect types are provided
    """
    # setting seed
    np.random.seed(seed)

    # type checking
    if not isinstance(target_col, str):
        raise TypeError("'target_col' must be a string type")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("'dataset' must be a pandas dataframe")

    # selecting column, shuffle values within column, add to dataframe
    dataset[target_col] = np.random.permutation(dataset[target_col].values)
    return dataset


def shuffle_features(feature_vals: np.array, seed: Optional[int] = 0) -> np.array:
    """suffles all values within feature space

    Parameters
    ----------
    feature_vals : np.array
        shuffled

    seed : Optional[int]
        setting random seed

    Returns
    -------
    np.array
        Returns shuffled values within the feature space

    Raises
    ------
    TypeError
        Raised if a numpy array is not provided
    """
    # setting seed
    np.random.seed(seed)

    # shuffle given array
    if not isinstance(feature_vals, np.ndarray):
        raise TypeError("'feature_vals' must be a numpy array")
    if feature_vals.ndim != 2:
        raise TypeError("'feature_vals' must be a 2x2 matrix")

    # creating a copy for feature vales to prevent overwriting of global variables
    feature_vals = np.copy(feature_vals)

    # shuffling feature space
    n_cols = feature_vals.shape[1]
    for col_idx in range(0, n_cols):
        # selecting column, shuffle, and update:
        feature_vals[:, col_idx] = np.random.permutation(feature_vals[:, col_idx])

    return feature_vals

## Setting up Paths and loading data

In [3]:
# load in the treatment groups
ground_truth = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
).resolve(strict=True)
# load in the ground truth
ground_truth = toml.load(ground_truth)
apoptosis_ground_truth = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis_ground_truth = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control_ground_truth = ground_truth["Healthy"]["healthy_groups_list"]

In [4]:
single_cell_data = pathlib.Path(
    f"../../data/PBMC_preprocessed_sc_norm_aggregated_nomic.parquet"
).resolve(strict=True)
df = pd.read_parquet(single_cell_data)

In [5]:
# out paths
map_out_dir = pathlib.Path("../data/processed/mAP_scores/secretome/")
map_out_dir.mkdir(exist_ok=True, parents=True)

# regular data output
# saving to csv
regular_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_regular_class.csv")

# shuffled data output
shuffled_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_shuffled_class.csv")

# shuffled feature space output
shuffled_feat_space_map_path = pathlib.Path(
    map_out_dir / "mAP_scores_shuffled_feature_space_class.csv"
)

### Clean up data

In [6]:
# add apoptosis, pyroptosis and healthy columns to dataframe
df["Apoptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in apoptosis_ground_truth,
    axis=1,
)
df["Pyroptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in pyroptosis_ground_truth,
    axis=1,
)
df["Control"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
df["Metadata_labels"] = df.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
# # drop apoptosis, pyroptosis, and healthy columns
df.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)
df.drop(columns=["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"], inplace=True)

In [7]:
# keep columns that contain Metdata and ['NSU']
df = df.filter(regex="Metadata|NSU")
df.head()

,Metadata_Well,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],...,uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],Metadata_labels
0,B02,0.799864,0.239781,0.771419,0.227135,0.282281,0.077979,0.517218,0.268053,0.205627,...,0.396902,0.385081,1.000000,0.000000,0.430111,0.538503,0.784695,0.468448,0.237545,Pyroptosis
1,B03,0.758205,0.661245,0.794392,0.712920,0.236378,0.288705,1.000000,0.314184,0.580188,...,0.256691,0.327491,0.390866,0.406489,0.412096,0.104830,0.812933,0.518536,0.244397,Pyroptosis
2,B04,0.128848,0.555580,0.823758,0.246641,0.249401,0.109274,0.844234,0.368186,0.506013,...,0.555221,0.357476,0.346884,0.477553,0.427658,0.642061,0.249380,0.627712,0.318350,Pyroptosis
3,B05,0.061989,0.510255,0.785998,0.615661,0.000000,0.251962,0.298381,0.492203,0.428714,...,0.308536,0.588899,0.828371,0.484102,0.294634,0.673648,0.236793,0.557634,0.350429,Pyroptosis
4,B06,0.097710,0.461685,0.270477,0.514695,0.479281,0.270494,0.708849,0.134432,0.350986,...,0.469875,0.395392,0.560129,0.504521,0.490444,0.258834,0.238358,0.524276,0.250670,Control


In [8]:
# output directories
map_out_dir = pathlib.Path("../data/processed/mAP_scores/")
map_out_dir.mkdir(parents=True, exist_ok=True)

### mAP Pipeline Parameters

The null size needs to be determined for the mAP pipeline. This is the size of the null class that is used to determine the mAP score.

In [9]:
tmp = (
    df.groupby(["Metadata_labels"])
    .count()
    .reset_index()[["Metadata_Well", "Metadata_labels"]]
)
# get the Pyroptosis number of Metadata_Well
Pyroptosis_count = tmp[tmp["Metadata_labels"] == "Pyroptosis"]["Metadata_Well"].values[
    0
]
Pyroptosis_count

65

In [10]:
pos_sameby = [
    "Metadata_labels",
]
pos_diffby = ["Metadata_Well"]

neg_sameby = []
neg_diffby = ["Metadata_labels"]

null_size = Pyroptosis_count
batch_size = 1

# number of resampling
n_resamples = 10

### mAP analysis for non-shuffled data

In [11]:
# This will generated 100 values [0..100] as seed values
# This will occur per phenotype

# spliting metadata and raw feature values
logging.info("splitting data set into metadata and raw feature values")
df_meta, df_feats = utils.split_data(df)
df_feats = np.array(df_feats)

# execute pipeline on negative control with training dataset with cp features
try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(f"Running pipeline on CP features using phenotype")
    result = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding columns
    result["shuffled"] = "non-shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotye:. Skipping")
# concatenating all datasets
result.to_csv(regular_feat_map_path, index=False)
result.head()

  0%|          | 0/4958 [00:00<?, ?it/s]

  0%|          | 0/6068 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,Pyroptosis,0.839120,0.015152,64,148,non-shuffled
1,B03,Pyroptosis,0.814110,0.015152,64,148,non-shuffled
2,B04,Pyroptosis,0.934453,0.015152,64,148,non-shuffled
3,B05,Pyroptosis,0.955751,0.015152,64,148,non-shuffled
4,B06,Control,0.946929,0.015152,75,148,non-shuffled


### mAP analysis for shuffled data (Phenotype)

In [12]:
logging.info("Running mAP pipeline with shuffled phenotype labeled data")
seed = 0
# running process
# for loop selects one single phenotype
# then splits the data into metadata and raw feature values
# two different groups that contains 3 splits caused by the types of features

# This will generated 100 values [0..100] as seed values
# splitting metadata labeled shuffled data
logging.info("splitting shuffled data set into metadata and raw feature values")
df = shuffle_meta_labels(dataset=df, target_col="Metadata_labels", seed=seed)
(
    df_meta,
    df_feats,
) = utils.split_data(df)


df_feats = np.array(df_feats)

try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(
        f"Running pipeline on CP features using  phenotype, data is shuffled by phenoptype labels"
    )
    shuffled_result = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding shuffle label column
    shuffled_result["shuffled"] = "phenotype_shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotye: Skipping")

# saving to csv
shuffled_result.to_csv(shuffled_feat_map_path, index=False)
shuffled_result

  0%|          | 0/4958 [00:00<?, ?it/s]

  0%|          | 0/6068 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,Control,0.550347,0.303030,75,148,phenotype_shuffled
1,B03,Control,0.549305,0.318182,75,148,phenotype_shuffled
2,B04,Control,0.568425,0.166667,75,148,phenotype_shuffled
3,B05,Control,0.562292,0.181818,75,148,phenotype_shuffled
4,B06,Pyroptosis,0.439118,0.712121,64,148,phenotype_shuffled
...,...,...,...,...,...,...,...
144,O07,Pyroptosis,0.431923,0.727273,64,148,phenotype_shuffled
145,O08,Apoptosis,0.047482,0.727273,7,148,phenotype_shuffled
146,O09,Pyroptosis,0.453693,0.590909,64,148,phenotype_shuffled
147,O10,Control,0.524448,0.530303,75,148,phenotype_shuffled


### mAP analysis for shuffled data (Feature space)

In [13]:
seed = 0


# split the shuffled dataset
# spliting metadata and raw feature values
logging.info("splitting shuffled data set into metadata and raw feature values")
(
    df_meta,
    df_feats,
) = utils.split_data(df)

df_feats = np.array(df_feats)


# shuffling the features, this will overwrite the generated feature space from above with the shuffled one
df_feats = shuffle_features(feature_vals=df_feats, seed=seed)


try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(
        f"Running pipeline on CP features using phenotype, feature space is shuffled"
    )
    shuffled_feat_map = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding shuffle label column
    shuffled_feat_map["shuffled"] = "features_shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotype:  Skipping")


# saving to csv
shuffled_feat_map.to_csv(shuffled_feat_space_map_path, index=False)
shuffled_feat_map

  0%|          | 0/4958 [00:00<?, ?it/s]

  0%|          | 0/6068 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,Control,0.582430,0.121212,75,148,features_shuffled
1,B03,Control,0.486050,0.833333,75,148,features_shuffled
2,B04,Control,0.492597,0.772727,75,148,features_shuffled
3,B05,Control,0.574753,0.136364,75,148,features_shuffled
4,B06,Pyroptosis,0.469231,0.393939,64,148,features_shuffled
...,...,...,...,...,...,...,...
144,O07,Pyroptosis,0.430981,0.727273,64,148,features_shuffled
145,O08,Apoptosis,0.095727,0.227273,7,148,features_shuffled
146,O09,Pyroptosis,0.418844,0.818182,64,148,features_shuffled
147,O10,Control,0.494031,0.727273,75,148,features_shuffled
